#                              Data Engineer Technical Test

In [1]:
#conda update jupyter
#%load_ext sql

In [2]:
#pip install psycopg2
#pip install sqlalchemy

<font color='blue'>Connecting to Postgresql Database</font>

In [3]:
import psycopg2
import pandas as pd
import sqlalchemy
from sqlalchemy import types
from sqlalchemy import create_engine

In [ ]:
try:
    conn = psycopg2.connect("dbname='mydb' user='postgres' host='localhost' password='Ashwini@12'")
except:
    print("Database Connection Error")
conn.autocommit = True
cursor = conn.cursor()

<font color='blue'>Loading CSV in Dataframe</font>

In [ ]:
df = pd.read_csv (r'C:\Users\Ashwini\Ably\transactions.csv')   


In [ ]:
df.head()

In [ ]:
df['timestamp'] = pd.to_datetime(df['timestamp'])

In [8]:
df['timestamp'] = df['timestamp'].dt.strftime('%d/%m/%Y')

In [9]:
print(df)

                           transaction_id   user   timestamp             tz  \
0    4afd43a4-7833-4883-aa4f-1506cba532ce  47735  02/02/2022   Europe/Paris   
1    1777cfe1-0ab7-4c57-94f7-99c58d8a419c  76257  11/01/2022   Europe/Paris   
2    f19e7c82-7b90-401d-a6af-fef1a7ecfbe7  30358  11/01/2022     US/Pacific   
3    d3e7b79b-fb99-434f-9a44-04df707f3741  24417  19/02/2022     US/Pacific   
4    fe816fbb-9f09-4bd7-bdb4-46dfdbf4888a  56415  09/02/2022     US/Pacific   
..                                    ...    ...         ...            ...   
865  69b5ff88-b2c7-4122-b560-74c968632e62  44868  11/01/2022     US/Eastern   
866  898881ce-d7fc-418b-9eb8-74d64d4a0a37  47819  20/01/2022     US/Eastern   
867  42aaae24-d50b-485b-a7ad-e58442aec87a  60535  20/01/2022  Europe/London   
868  539759c5-5303-4e48-b2a8-d6ba4797b15c  62664  13/01/2022     US/Eastern   
869  14235c77-8ae6-4b0c-8626-ca8f0b86f97a  94520  08/02/2022     US/Pacific   

       currency      value  cleared        country 

<font color='blue'>Loading data into the Database</font>

In [ ]:
cursor.execute("DROP TABLE IF EXISTS transaction_raw")

In [ ]:
engine = create_engine('postgresql://postgres:Ashwini@12@localhost/mydb')
df.to_sql('transaction_raw',con= engine,if_exists='replace',index=False,
           dtype ={'transaction_id': sqlalchemy.types.VARCHAR(50),
                     'user' : sqlalchemy.types.INTEGER,
                     'timestamp' :sqlalchemy.types.Date,
                     'tz' :sqlalchemy.types.VARCHAR(length=20),
                     'currency':sqlalchemy.types.VARCHAR(length=10),
                     'value' :sqlalchemy.types.Numeric,
                     'cleared' :sqlalchemy.types.Boolean,
                     'country':sqlalchemy.types.VARCHAR(length=50)})

In [ ]:
cursor.execute("DROP TABLE IF EXISTS transaction")
sql="""Create Table transaction as 
        Select * from transaction_raw
        where value is not null"""
cursor.execute(sql)

<font color='blue'>Transforming the data</font>

In [ ]:
sql = """SELECT table_name, column_name, data_type 
         FROM information_schema.columns
         WHERE table_name = 'transaction'"""
cursor.execute(sql)
for i in cursor.fetchall():
    print(i)

In [ ]:
s = """ALTER TABLE transaction
       ALTER COLUMN value TYPE Numeric(10,2);
       """
cursor.execute(s)

In [ ]:
up = """UPDATE transaction
        SET currency = 'EUR' 
        WHERE currency IN ('Euro')"""
cursor.execute(up)
upd = """UPDATE transaction
        SET currency = 'USD' 
        WHERE currency IN ('US Dollars')"""
cursor.execute(upd)

In [ ]:
s0 = """UPDATE transaction
        SET currency = 'GBP' 
        WHERE currency isnull"""
cursor.execute(s0)

In [ ]:
s2 = """UPDATE transaction
        SET country = 'United Kingdom'
        where country is null"""
cursor.execute(s2)

s3 = """select * from transaction where country is null
        """
cursor.execute(s3)
for i in cursor.fetchall():
    print(i)

<font color='blue'>Extracting from API</font>

In [ ]:
import json,requests

In [ ]:
response = requests.get('https://v6.exchangerate-api.com/v6/435a57dc789ddcb46282a1b6/latest/GBP')

In [ ]:
c_rates = response.text

In [ ]:
rates_d = json.loads(c_rates)

In [ ]:
rates_d

In [ ]:
cursor.execute("DROP TABLE IF EXISTS exchange_rate")

In [ ]:
sql4 = """create table exchange_rate(currency varchar, rate Numeric(10,2))"""
cursor.execute(sql4)

In [ ]:
sql6 = """INSERT INTO exchange_rate VALUES(%s, %s)"""

for record in rates_d["conversion_rates"].items():
    cursor.execute(sql6, [record[0], record[1]])


In [ ]:
sql9 = """select * from exchange_rate """
cursor.execute(sql9)
for i in cursor.fetchall():
    print(i)

In [ ]:
sql10 = """SELECT table_name, column_name, data_type 
           FROM information_schema.columns
           WHERE table_name = 'exchange_rate'"""
cursor.execute(sql10)
for i in cursor.fetchall():
    print(i)